In [0]:
pip install tensorflow==2.1.0-rc0

In [0]:
pip install keras==2.3.1

In [0]:
import pandas as pd
import numpy as np
import pandas as pd
df=pd.read_csv('iris.csv')
df.head()
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['Species']=le.fit_transform(df['Species'])
df=df.drop(['Unnamed: 0'],1)
df=pd.concat([df.drop(['Species'],1),pd.get_dummies(df['Species'])],1)
df.head()
from keras.layers import Dense,Input
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.optimizers import SGD
inp=Input((4,))
d1=Dense(5,activation='relu')(inp)
d2=Dense(10,activation='relu')(d1)
d3=Dense(1,activation='sigmoid')(d2)
mod=Model(inputs=inp,output=d3)
mod.compile(loss='binary_crossentropy',optimizer=SGD(0.1))
mod.fit(StandardScaler().fit_transform(df[['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width']]),df[0],epochs=10)
pre=mod.predict(df[['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width']])
print(np.min(pre))
print(np.max(pre))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Epoch 1/10
150/150 [==============================] - 0s 491us/step - loss: 0.7686
Epoch 2/10
150/150 [==============================] - 0s 55us/step - loss: 0.5669
Epoch 3/10
150/150 [==============================] - 0s 66us/step - loss: 0.4861
Epoch 4/10
150/150 [==============================] - 0s 62us/step - loss: 0.4282
Epoch 5/10
150/150 [==============================] - 0s 73us/step - loss: 0.3803
Epoch 6/10
150/150 [==============================] - 0s 64us/step - loss: 0.3335
Epoch 7/10
150/150 [==============================] - 0s 64us/step - loss: 0.2828
Epoch 8/10
150/150 [==============================] - 0s 107us/step - loss: 0.2347
Epoch 9/10
150/150 [==============================] - 0s 93us/step - loss: 0.1928
Epoch 10/10
150/150 [==============================] - 0s 83us/step - loss: 0.1589
0.0
0.000497669


In [0]:
import numpy as np
from sklearn.metrics import log_loss
def cross(a,b):
    itr1=a.shape[0]
    itr2=b.shape[1]
    ans=np.zeros((a.shape[0],b.shape[1]))
    for i in range(itr1):
        for j in range(itr2):
            cum=0
            for k in range(a.shape[1]):
                cum+=a[i,k]*b[k,j]
            ans[i,j]=cum
    ans[ans<0]=0
    return ans
def dense(inp,out,act='linear'):
    slopes=np.random.uniform(size=(inp.shape[1],out))*(2/inp.shape[1])**0.5
    coef=np.random.uniform(size=(1,out))*(2/inp.shape[1])**0.5
    pred=cross(inp,slopes)+coef
    shp=out
    out=pred
    if act=='sigmoid':
        out = np.exp(out*-1)+1
        out = 1/out
    elif act=='relu':
        print(act)
        out[out<0]=0
    return [out,slopes,coef,inp,np.zeros((1,shp))[0],np.zeros_like(slopes),np.zeros((1,shp))[0],np.zeros_like(slopes),act]
def fwd(ans):
    for en,lyr in enumerate(ans):
        lyr[0]=cross(lyr[3],lyr[1])
        
        lyr[0]+=lyr[2]
        if lyr[-1]=='relu':
            lyr[0][lyr[0]<0]=0
        elif lyr[-1]=='sigmoid':
            lyr[0] = np.exp(lyr[0]*-1)+1
            lyr[0] = 1/lyr[0]
        try:
            ans[en+1][3]=lyr[0]
        except:
            continue
    
    return ans[::-1]
def bce(pred,true,typ=0):
    pos=true[true==1]
    pos_p=pred[true==1]
    neg=true[true==0]
    neg_p=pred[true==0]
    if typ==1:
        return log_loss(true,pred)
    else:
        ls=[]
        pos=[]
        neg=[]
        for e,i in enumerate(pred):
            if true[e]==1:
                ls.append(-1/i)
                pos.append(1/i)
            else:
                ls.append(1/(1-i))
                neg.append(1/(1-i))
        return np.asarray(ls)
def disp(ans):
    for i in ans:
        for j in range(7):
            print(i[j].shape)
        print('*'*20)
import seaborn as sns
from matplotlib import pyplot as plt
def bck(ans,epochs,lr,y,B1):
    #[dns3,dns2,dns1]
    VDB=0
    VDW=0
    SDB=0
    SDW=0
    for i in range(epochs):
        for en,lyr in enumerate(ans):
            grad=bce(ans[0][0],y)
            for i in range(0,en):
                grad=cross(grad,ans[i][1].T)
            lyr[4]=lyr[4]*B1+grad*(1-B1)
            lyr[2]=lyr[2]-(lyr[4]*lr)
            print('*'*20)
            grad=cross(lyr[3].T,grad)
            grad=grad/len(y)
            lyr[5]=lyr[5]*B1+grad*(1-B1)
            lyr[1]=lyr[1]-(lyr[5]*lr)
            ans=fwd(ans[::-1])
            
        print(bce(np.asarray(ans[0][0]),y,1))
        print('one epoch finished')
        print('#'*20)
import pandas as pd
from sklearn.preprocessing import StandardScaler
df=pd.read_csv('iris.csv')
df.head()
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['Species']=le.fit_transform(df['Species'])
df=df.drop(['Unnamed: 0'],1)
df=pd.concat([df.drop(['Species'],1),pd.get_dummies(df['Species'])],1)
df.head()
d1=dense(StandardScaler().fit_transform(df[['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width']].values),5,'relu')
d2=dense(d1[0],10,'relu')
d3=dense(d2[0],1,'sigmoid')
bck([d3,d2,d1],10,0.1,np.asarray(df[0]),0.9)

relu
relu
********************
********************
********************
0.626804570169977
one epoch finished
####################
********************
********************
********************
0.5648416425104491
one epoch finished
####################
********************
********************
********************
0.5142678122167015
one epoch finished
####################
********************
********************
********************
0.4696962636366827
one epoch finished
####################
********************
********************
********************
0.4287614850684823
one epoch finished
####################
********************
********************
********************
0.39051782069028057
one epoch finished
####################
********************
********************
********************
0.35464158516734623
one epoch finished
####################
********************
********************
********************
0.32106321680872185
one epoch finished
####################
************

In [0]:
print(max(list(d3[0].reshape(1,-1)[0])))
print(min(list(d3[0].reshape(1,-1)[0])))

0.7004683999866805
5.933897491144277e-45
